In [3]:
import pandas as pd
train=pd.read_csv('file_mapping.csv')
train.head()


,Filename,Brand Name,Label,Tagline
0,output\Cowbell\000001.jpg,Cowbell,Genuine,sooo creamy sooo gooood
1,genLogoOutput\Rolex\000003.jpg,Rolex,Fake,antiophthalmic_factor crown for every accompli...
2,output\PlayStation\000005.jpg,PlayStation,Genuine,eNoS Lives.
3,output\YouTube\000004.jpg,YouTube,Genuine,Broadcast Yourself
4,genLogoOutput\Lays\000004.jpg,Lays,Fake,eNoS sprightliness .


In [4]:
train=train[['Label','Tagline']]
train.head()

,Label,Tagline
0,Genuine,sooo creamy sooo gooood
1,Fake,antiophthalmic_factor crown for every accompli...
2,Genuine,eNoS Lives.
3,Genuine,Broadcast Yourself
4,Fake,eNoS sprightliness .


In [5]:
train.reset_index(inplace=True)
train.columns=['ID','class','text']
train.head()

,ID,class,text
0,0,Genuine,sooo creamy sooo gooood
1,1,Fake,antiophthalmic_factor crown for every accompli...
2,2,Genuine,eNoS Lives.
3,3,Genuine,Broadcast Yourself
4,4,Fake,eNoS sprightliness .


In [7]:
import os
import re

def clean_text(text):
    if isinstance(text, str):
        cleaned_text = text.lower()  # Convert to lowercase
        cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)  # Remove punctuation
        return cleaned_text
    else:
        return str(text)

def create_files(df, folder_path):
    """
    Create files from a dataframe where each row is saved as a text file named after the ID column.
    The text file contains the cleaned value in the text column.
    
    Args:
        df (pandas.DataFrame): A dataframe with columns 'ID', 'text', and 'target'
        folder_path (str): A string indicating the path to the folder where the files will be stored.
        
    Returns:
        None
    """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    for index, row in df.iterrows():
        ID_value = row['ID']
        text_value = row['text']
        
        cleaned_text = clean_text(text_value)
        
        file_name = os.path.join(folder_path, f"{ID_value}.txt")
        
        with open(file_name, 'w') as f:
            f.write(str(cleaned_text))

In [8]:
create_files(train, './data')

In [9]:
train['class'].value_counts()

Fake       550
Genuine    275
Name: class, dtype: int64

In [10]:
round(train['class'].value_counts()*100/len(train),2)

Fake       66.67
Genuine    33.33
Name: class, dtype: float64

In [11]:
train.head()

,ID,class,text
0,0,Genuine,sooo creamy sooo gooood
1,1,Fake,antiophthalmic_factor crown for every accompli...
2,2,Genuine,eNoS Lives.
3,3,Genuine,Broadcast Yourself
4,4,Fake,eNoS sprightliness .


In [12]:
train['filename']='./data/'+train['ID'].astype('str')+'.txt'
train=train.drop(['ID','text'],axis=1)
train.columns=['label','filename']
train.head()

,label,filename
0,Genuine,./data/0.txt
1,Fake,./data/1.txt
2,Genuine,./data/2.txt
3,Genuine,./data/3.txt
4,Fake,./data/4.txt


In [13]:
from sklearn.model_selection import train_test_split
train,test= train_test_split( train, test_size=0.2, random_state=42,shuffle=True, stratify=train['label'])

In [14]:
count=(train.label.value_counts()).to_frame()
count.reset_index(inplace=True)
count.columns=['label','count']
count

,label,count
0,Fake,440
1,Genuine,220


In [15]:
train=train.sample(n=len(train))
train.reset_index(inplace=True,drop=True)
train.head()

,label,filename
0,Fake,./data/207.txt
1,Fake,./data/570.txt
2,Fake,./data/210.txt
3,Fake,./data/164.txt
4,Fake,./data/427.txt


In [16]:
test=test.sample(n=len(test))
test.reset_index(inplace=True,drop=True)
test.head()

,label,filename
0,Fake,./data/52.txt
1,Fake,./data/680.txt
2,Fake,./data/341.txt
3,Fake,./data/496.txt
4,Genuine,./data/498.txt


In [17]:
train.shape,test.shape

((660, 2), (165, 2))

In [18]:
import os
newpath = r'./train' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [19]:
class_names = list(train.label.unique())
class_names[:5]

['Fake', 'Genuine']

In [20]:
import shutil, os
#creating subfolders
for c in class_names:
    dest =  r'./train/'+str(c)
    os.makedirs(dest)
    for i in list(train[train['label']==c]['filename']): # Image Id
        move_image_to_cat = shutil.copy(i, dest)

In [21]:
newpath = r'./test' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [22]:
import shutil, os
#creating subfolders
for c in class_names:
    dest =  r'./test/'+str(c)
    os.makedirs(dest)
    for i in list(test[test['label']==c]['filename']): # Image Id
        move_image_to_cat = shutil.copy(i, dest)

In [23]:
import shutil
shutil.rmtree('./data')